In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [29]:
RATING_PATH = '../data/raw/Number_of_ratings.xlsx'
DEFAULT_PATH = '../data/raw/Number_of_defaults.xlsx'

ratings = pd.read_excel(RATING_PATH).T
defaults = pd.read_excel(DEFAULT_PATH).T

ratings = ratings.iloc[1:, [7, 6, 5, 4, 3, 2, 1]]
ratings.to_csv("../data/processed/ratings.csv", index=False, header=False)

defaults = defaults.iloc[1:, [7, 6, 5, 4, 3, 2, 1]]
defaults.to_csv("../data/processed/defaults.csv", index=False, header=False)
ratings

,7,6,5,4,3,2,1
Number of ratings,154,1471,1047,1637,1330,350,24
Unnamed: 2,158,1629,1078,1689,1286,321,21
Unnamed: 3,182,1813,1168,1708,1290,325,19
Unnamed: 4,202,1927,1284,1797,1348,354,16
Unnamed: 5,195,1930,1324,1813,1383,327,9
Unnamed: 6,203,2081,1343,1870,1400,326,8
Unnamed: 7,238,2078,1289,1855,1432,322,8
Unnamed: 8,455,1939,1184,1836,1405,290,8
Unnamed: 9,318,2110,1276,1893,1394,275,8
Unnamed: 10,327,1935,1200,1859,136,282,7


In [5]:
ratings = pd.read_csv("../data/processed/ratings.csv", header=None)
defaults = pd.read_csv("../data/processed/defaults.csv", header=None)

# Convert to numpy arrays
N_tk = ratings.values
n_tk = defaults.values

T, K = N_tk.shape

ratings = ['CCC', 'B', 'BB', 'BBB', 'A', 'AA', 'AAA']

# x_t
x_t = np.array([2.7, 3.9, 3.1, 2.8, 3.3, 2.7, -2.9, 6.4, 3.2, 2.8]).reshape(1, )
x_t.shape


In [7]:
import numpy as np
from scipy.stats import norm, gamma
import matplotlib.pyplot as plt

def logit(p):
    return 1 / (1 + np.exp(-p))

# Gibbs Sampler for Model 3
def gibbs_model3(N_tk, n_tk, x_t, n_iter=10000, burn_in=1000):
    T, K = N_tk.shape

    # Initial values
    mu = np.linspace(-2, -10, K)  # Initial guess: decreasing from -2 to -10
    beta = np.random.uniform(-0.5, 0.5, K)  # Initial guess for beta_k
    phi = np.full(K, 0.5)  # Positive initial phi_k
    alpha = 0.5
    b = np.random.normal(0, 1, T)  # Initialize b with variation
    tau = 100.0
    eta, v = 2, 1  # Proper inverse-gamma prior for phi_k^2

    # Storage
    mu_trace = np.zeros((n_iter, K))
    beta_trace = np.zeros((n_iter, K))
    phi_trace = np.zeros((n_iter, K))
    alpha_trace = np.zeros(n_iter)
    b_trace = np.zeros((n_iter, T))

    # Likelihood and prior functions
    def log_likelihood_mu_k(k, mu_k, beta_k, phi_k, b):
        ll = 0
        for t in range(T):
            eta = mu_k - x_t[t] * beta_k - phi_k * b[t]
            p = logit(eta)
            ll += N_tk[t, k] * np.log(p + 1e-10) + (n_tk[t, k] - N_tk[t, k]) * np.log(1 - p + 1e-10)
        return ll + norm.logpdf(mu_k, 0, tau)

    def log_likelihood_beta_k(k, mu_k, beta_k, phi_k, b):
        ll = 0
        for t in range(T):
            p = logit(mu_k - x_t[t] * beta_k - phi_k * b[t])
            ll += N_tk[t, k] * np.log(p + 1e-10) + (n_tk[t, k] - N_tk[t, k]) * np.log(1 - p + 1e-10)
        return ll + norm.logpdf(beta_k, 0, tau)

    def log_likelihood_phi_k(k, mu_k, beta_k, phi_k, b):
        ll = 0
        for t in range(T):
            p = logit(mu_k - x_t[t] * beta_k - phi_k * b[t])
            ll += N_tk[t, k] * np.log(p + 1e-10) + (n_tk[t, k] - N_tk[t, k]) * np.log(1 - p + 1e-10)
        return ll - (eta + 1) * np.log(phi_k**2 + 1e-10) - v / (phi_k**2 + 1e-10) if phi_k > 0 else -np.inf

    def log_likelihood_b_t_likelihood_only(t, b_t, mu, beta, phi):
        ll = 0
        for k in range(K):
            p = logit(mu[k] - x_t[t] * beta[k] - phi[k] * b_t)
            ll += N_tk[t, k] * np.log(p + 1e-10) + (n_tk[t, k] - N_tk[t, k]) * np.log(1 - p + 1e-10)
        return ll

    def log_likelihood_alpha(b, alpha):
        if not (-1 < alpha < 1):
            return -np.inf
        Sigma_inv = np.zeros((T, T))
        for i in range(T):
            Sigma_inv[i, i] = 1 + alpha**2 if 0 < i < T-1 else 1
            if i < T-1:
                Sigma_inv[i, i+1] = Sigma_inv[i+1, i] = -alpha
        det_Sigma_inv = (1 - alpha**2 + 1e-10)
        ll = 0.5 * np.log(det_Sigma_inv) - 0.5 * b @ Sigma_inv @ b
        return ll

    # Gibbs sampling loop
    for it in range(n_iter):
        # Update mu_k
        for k in range(K):
            mu_prop = mu[k] + np.random.normal(0, 0.5)
            mu_temp = mu.copy()
            mu_temp[k] = mu_prop
            if all(mu_temp[j] > mu_temp[j+1] for j in range(K-1)):
                log_ratio = log_likelihood_mu_k(k, mu_prop, beta[k], phi[k], b) - \
                            log_likelihood_mu_k(k, mu[k], beta[k], phi[k], b)
                if np.isfinite(log_ratio) and np.log(np.random.rand()) < log_ratio:
                    mu[k] = mu_prop

        # Update beta_k
        for k in range(K):
            beta_prop = beta[k] + np.random.normal(0, 0.1)
            log_ratio = log_likelihood_beta_k(k, mu[k], beta_prop, phi[k], b) - \
                        log_likelihood_beta_k(k, mu[k], beta[k], phi[k], b)
            if np.isfinite(log_ratio) and np.log(np.random.rand()) < log_ratio:
                beta[k] = beta_prop

        # Update phi_k
        for k in range(K):
            phi_prop = phi[k] + np.random.normal(0, 0.1)
            if phi_prop > 0:
                log_ratio = log_likelihood_phi_k(k, mu[k], beta[k], phi_prop, b) - \
                            log_likelihood_phi_k(k, mu[k], beta[k], phi[k], b)
                if np.isfinite(log_ratio) and np.log(np.random.rand()) < log_ratio:
                    phi[k] = phi_prop

        # Update b_t with MH using prior as proposal
        for t in range(T):
            b_prev = b[t-1] if t > 0 else 0
            b_next = b[t+1] if t < T-1 else 0
            if t == 0:
                mean_prior = alpha * b_next
                var_prior = (1 + alpha**2) / (1 - alpha**2 + 1e-10)
                b_prop = np.random.normal(mean_prior, np.sqrt(var_prior))
                log_prior_current = norm.logpdf(b[t], 0, 1 / np.sqrt(1 - alpha**2 + 1e-10)) + \
                                   norm.logpdf(b_next, alpha * b[t], 1)
                log_prior_prop = norm.logpdf(b_prop, 0, 1 / np.sqrt(1 - alpha**2 + 1e-10)) + \
                                 norm.logpdf(b_next, alpha * b_prop, 1)
            elif t == T-1:
                mean_prior = alpha * b_prev
                var_prior = 1
                b_prop = np.random.normal(mean_prior, np.sqrt(var_prior))
                log_prior_current = norm.logpdf(b[t], alpha * b_prev, 1)
                log_prior_prop = norm.logpdf(b_prop, alpha * b_prev, 1)
            else:
                mean_prior = alpha * (b_prev + b_next) / (1 + alpha**2 + 1e-10)
                var_prior = 1 / (1 + alpha**2 + 1e-10)
                b_prop = np.random.normal(mean_prior, np.sqrt(var_prior))
                log_prior_current = norm.logpdf(b[t], alpha * b_prev, 1) + \
                                   norm.logpdf(b_next, alpha * b[t], 1)
                log_prior_prop = norm.logpdf(b_prop, alpha * b_prev, 1) + \
                                 norm.logpdf(b_next, alpha * b_prop, 1)

            log_likelihood_current = log_likelihood_b_t_likelihood_only(t, b[t], mu, beta, phi)
            log_likelihood_prop = log_likelihood_b_t_likelihood_only(t, b_prop, mu, beta, phi)
            log_ratio = log_likelihood_prop - log_likelihood_current + \
                        log_prior_current - log_prior_prop
            if np.isfinite(log_ratio) and np.log(np.random.rand()) < log_ratio:
                b[t] = b_prop

        # Update alpha
        while True:
            alpha_prop = np.random.uniform(-1, 1)
            log_ratio = log_likelihood_alpha(b, alpha_prop) - log_likelihood_alpha(b, alpha)
            if np.isfinite(log_ratio) and np.log(np.random.rand()) < log_ratio:
                alpha = alpha_prop
                break

        # Store traces
        mu_trace[it] = mu
        beta_trace[it] = beta
        phi_trace[it] = phi
        alpha_trace[it] = alpha
        b_trace[it] = b

    return {
        'mu': mu_trace[burn_in:],
        'beta': beta_trace[burn_in:],
        'phi': phi_trace[burn_in:],
        'alpha': alpha_trace[burn_in:],
        'b': b_trace[burn_in:]
    }

# Run Model 3
results_model3 = gibbs_model3(N_tk, n_tk, x_t)
print("Model 3 Estimates:")
print("mu:", np.mean(results_model3['mu'], axis=0))
print("beta:", np.mean(results_model3['beta'], axis=0))
print("phi:", np.mean(results_model3['phi'], axis=0))
print("alpha:", np.mean(results_model3['alpha']))

# Visualization
plt.figure(figsize=(15, 10))

# mu traces
plt.subplot(2, 2, 1)
for k in range(K):
    plt.plot(results_model3['mu'][:, k], label=f'mu_{ratings[k]}')
plt.legend()
plt.title('Trace Plots for mu')
plt.xlabel('Iteration')
plt.ylabel('Value')

# beta traces
plt.subplot(2, 2, 2)
for k in range(K):
    plt.plot(results_model3['beta'][:, k], label=f'beta_{ratings[k]}')
plt.legend()
plt.title('Trace Plots for beta')
plt.xlabel('Iteration')
plt.ylabel('Value')

# phi traces
plt.subplot(2, 2, 3)
for k in range(K):
    plt.plot(results_model3['phi'][:, k], label=f'phi_{ratings[k]}')
plt.legend()
plt.title('Trace Plots for phi')
plt.xlabel('Iteration')
plt.ylabel('Value')

# alpha trace
plt.subplot(2, 2, 4)
plt.plot(results_model3['alpha'], label='alpha')
plt.legend()
plt.title('Trace Plot for alpha')
plt.xlabel('Iteration')
plt.ylabel('Value')

plt.tight_layout()
plt.show()

# Optional: Plot b_t for a few time points
plt.figure(figsize=(10, 6))
for t in [0, 10, 20, 30]:
    plt.plot(results_model3['b'][:, t], label=f'b_{t}')
plt.legend()
plt.title('Trace Plots for b_t (Selected Time Points)')
plt.xlabel('Iteration')
plt.ylabel('Value')
plt.show()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()